# Drone Data Interactive Visualization

This notebook creates an interactive map for visualizing drone data with date and time selection capabilities.

In [2]:
import pandas as pd
import json

df_orig = pd.read_csv('Dataset/cleaned_drone_data.csv')
df_pred = pd.read_csv('Dataset/drone_weather_predictions.csv')

if 'YEAR' not in df_orig.columns and 'FlightDate' in df_orig.columns:
    df_orig['YEAR'] = pd.to_datetime(df_orig['FlightDate']).dt.year
if 'YEAR' not in df_pred.columns and 'FlightDate' in df_pred.columns:
    df_pred['YEAR'] = pd.to_datetime(df_pred['FlightDate']).dt.year

df_orig['FlightDate'] = pd.to_datetime(df_orig['FlightDate'])
df_orig['formatted_date'] = df_orig['FlightDate'].dt.strftime('%Y-%m-%d')
df_pred['FlightDate'] = pd.to_datetime(df_pred['FlightDate'])
df_pred['formatted_date'] = df_pred['FlightDate'].dt.strftime('%Y-%m-%d')

df = pd.merge(
    df_orig,
    df_pred[['FlightDate', 'FIELDNUMBER', 'LOCATION', 'VARIETY', 'YEAR', 'Predicted_Height', 'Predicted_Cover']],
    on=['FlightDate', 'FIELDNUMBER', 'LOCATION', 'VARIETY', 'YEAR'],
    how='left'
)

for col in ['formatted_date', 'LOCATION', 'VARIETY', 'FIELDNUMBER']:
    df[col] = df[col].astype(str)

dates = sorted(df['formatted_date'].unique())
varieties = sorted(df['VARIETY'].unique())

min_date = min(dates)
max_date = max(dates)

frontend_cols = [
    'formatted_date', 'LOCATION', 'VARIETY', 'FIELDNUMBER', 'YEAR',
    'VegetationHeight_p75', 'VegetationCover_perc',
    'Predicted_Height', 'Predicted_Cover'
]
data_json = df[frontend_cols].to_json(orient='records')
dates_json = json.dumps(dates)
# locations_json = json.dumps(locations)
varieties_json = json.dumps(varieties)

In [3]:
import json

locations = sorted(df['LOCATION'].str.upper().unique())

location_to_municipality = {
    "RILLAND": "REIMERSWAAL",
    "EMMELOORD": "NOORDOOSTPOLDER"
}
municipalities = [location_to_municipality[loc] for loc in locations if loc in location_to_municipality]
municipalities_json = json.dumps([m.upper() for m in municipalities])

color_palette = [
    "#1abc9c", "#e67e22", "#3498db", "#9b59b6", "#e74c3c", "#f1c40f", "#34495e", "#16a085"
]
location_colors = {loc: color_palette[i % len(color_palette)] for i, loc in enumerate(locations)}
location_colors_json = json.dumps(location_colors)

frontend_cols = [
    'formatted_date', 'LOCATION', 'VARIETY', 'FIELDNUMBER',
    'VegetationHeight_p75', 'Predicted_Height', 'VegetationCover_perc', 'Predicted_Cover'
]
frontend_cols = [col for col in frontend_cols if col in df.columns]
data_json = df[frontend_cols].to_dict(orient='records')
data_json_str = json.dumps(data_json)
locations_json = json.dumps(sorted(df['LOCATION'].unique()))
varieties_json = json.dumps(sorted(df['VARIETY'].unique()))
dates_json = json.dumps(sorted(df['formatted_date'].unique()))
min_date = df['formatted_date'].min()
max_date = df['formatted_date'].max()

html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Crop Field Visualization</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/flatpickr/dist/flatpickr.min.css">
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/flatpickr/dist/themes/material_green.css">
    <link rel="stylesheet" href="https://unpkg.com/leaflet/dist/leaflet.css"/>
    <style>
        body {{
            font-family: 'Segoe UI', Arial, sans-serif;
            background: #f4f7fa;
            margin: 0;
            padding: 0;
        }}
        .top-bar {{
            display: flex;
            justify-content: center;
            align-items: flex-start;
            gap: 32px;
            margin-top: 30px;
            margin-bottom: 10px;
        }}
        #controls {{
            padding: 18px 24px 18px 24px;
            background: #fff;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.07);
            width: 320px;
        }}
        #controls h3 {{
            margin-top: 0;
            margin-bottom: 12px;
            font-size: 1.2em;
            color: #2d5c2d;
        }}
        #controls label {{
            font-weight: 500;
            color: #444;
        }}
        .select-row {{
            display: flex;
            flex-direction: column;
            margin-bottom: 14px;
        }}
        #controls select {{
            border-radius: 6px;
            border: 1px solid #b6b6b6;
            padding: 8px 10px;
            width: 100%;
            font-size: 1em;
            background: #f9f9f9;
            box-sizing: border-box;
        }}
        .calendar-row {{
            display: flex;
            flex-direction: column;
            align-items: flex-start;
            gap: 0;
            margin-bottom: 14px;
        }}
        #calendar-btn {{
            background: #f9f9f9;
            border: 1.5px solid #b6b6b6;
            border-radius: 50%;
            width: 40px;
            height: 40px;
            cursor: pointer;
            display: flex;
            align-items: center;
            justify-content: center;
            transition: background 0.2s, box-shadow 0.2s, border-color 0.2s;
            outline: none;
            font-size: 1.3em;
            padding: 0;
            margin-bottom: 8px;
        }}
        #calendar-btn svg {{
            width: 22px;
            height: 22px;
            display: block;
        }}
        #calendar-btn:hover, #calendar-btn:focus {{
            background: #e0f7e0;
            border-color: #4CAF50;
            box-shadow: 0 0 0 2px #4CAF5033;
        }}
        #date {{
            position: absolute;
            left: 0;
            top: 0;
            width: 1px;
            height: 1px;
            opacity: 0;
            pointer-events: none;
        }}
        #date-display {{
            font-size: 1em;
            color: #333;
            min-width: 110px;
            padding: 4px 0;
            margin-left: 0;
        }}
        #legend {{
            padding: 12px 16px;
            background: #fff;
            border-radius: 8px;
            box-shadow: 0 1px 8px rgba(0,0,0,0.06);
            width: 320px;
            font-size: 1em;
        }}
        #legend h3{{
        padding: 0px;
        margin:0px 0px 22px;
        }}
        #legend-table {{
            width: 100%;
            border-collapse: collapse;
        }}
        #legend-table td {{
            padding: 4px 8px;
            vertical-align: middle;
        }}
        #legend-table .emoji {{
            font-size: 1.3em;
            text-align: center;
            width: 36px;
        }}
        #field-visualization {{
            margin: 30px auto 30px auto;
            display: flex;
            flex-wrap: wrap;
            justify-content: space-evenly;
            align-items: flex-start;
            background: #e0e0b8;
            padding: 30px 0;
            margin-top: 100px;
            border-radius: 18px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.07);
            width: 90vw;
            min-height: 100px;
            max-width: 1400px;
        }}
        .crop-cell {{
            width: 110px;
            height: 110px;
            background: #d6eac7;
            border: 2px solid #a2a27a;
            border-radius: 12px;
            margin: 0 32px 32px 0;
            position: relative;
            cursor: pointer;
            transition: box-shadow 0.2s, transform 0.2s;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
        }}
        .crop-cell:last-child {{
            margin-right: 0;
        }}
        .crop-emoji {{
            font-size: 2.1em;
            margin-bottom: 2px;
        }}
        .crop-label {{
            font-size: 1em;
            color: #333;
            margin-top: 2px;
            word-break: break-all;
        }}
        .tooltip {{
            display: none;
            position: absolute;
            left: 50%;
            bottom: 110%;
            transform: translateX(-50%);
            background: #fff;
            color: #222;
            border: 1.2px solid #388e3c;
            border-radius: 6px;
            padding: 4px 10px;
            font-size: 0.92em;
            min-width: 150px;
            max-width: 200px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.13);
            white-space: pre-line;
            pointer-events: none;
            opacity: 0;
            transition: opacity 0.18s;
            z-index: 10;
            box-shadow: 0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19);
        }}
        .tooltip::before {{
            content: "";
            position: absolute;
            left: 50%;
            bottom: -8px;
            transform: translateX(-50%);
            border-width: 8px 7px 0 7px;
            border-style: solid;
            border-color: #388e3c transparent transparent transparent;
        }}
        .crop-cell:hover .tooltip {{
            display: block;
            opacity: 1;
        }}
        .flatpickr-calendar {{
            font-size: 1em;
        }}
        .flatpickr-day.selected, .flatpickr-day.startRange, .flatpickr-day.endRange, .flatpickr-day.selected.inRange, .flatpickr-day.startRange.inRange, .flatpickr-day.endRange.inRange {{
            background: #4CAF50;
            border-color: #388e3c;
        }}
        .flatpickr-day.today {{
            border-color: #4CAF50;
        }}
        #map-container {{
            position: fixed;
            top: 0;
            left: 0;
            width: 100vw;
            height: 100vh;
            z-index: 1;
            margin: 0;
            border-radius: 0;
            box-shadow: none;
        }}
        #back-btn {{
            display: none;
            position: fixed;
            top: 32px;
            left: 32px;
            z-index: 1000;
            background: linear-gradient(90deg, #4CAF50 0%, #388e3c 100%);
            color: #fff;
            border: none;
            border-radius: 24px;
            padding: 12px 28px;
            font-size: 1.1em;
            font-weight: 600;
            box-shadow: 0 2px 8px rgba(60,60,60,0.10);
            cursor: pointer;
            transition: background 0.2s, box-shadow 0.2s, transform 0.1s;
            outline: none;
            #back-btn:hover, #back-btn:focus {{
            background: linear-gradient(90deg, #388e3c 0%, #4CAF50 100%);
            box-shadow: 0 4px 16px rgba(60,60,60,0.13);
            transform: translateY(-2px) scale(1.03);
        }}
    </style>
</head>
<body>
    <div id="map-container"></div>
    <button id="back-btn">Back to map</button>
    <div class="top-bar" id="top-bar" style="display:none;">
        <div id="controls">
            <h3 id="location-title"></h3>
            <div class="select-row">
                <label for="variety-select">Variety:</label>
                <select id="variety-select"></select>
            </div>
            <div class="select-row calendar-row">
                <label for="date-select">Date:</label>
                <button id="calendar-btn" title="Pick a date" tabindex="0">
                    <svg viewBox="0 0 24 24" fill="none">
                        <rect x="3" y="5" width="18" height="16" rx="2" fill="#fff" stroke="#388e3c" stroke-width="2"/>
                        <rect x="7" y="2" width="2" height="4" rx="1" fill="#388e3c"/>
                        <rect x="15" y="2" width="2" height="4" rx="1" fill="#388e3c"/>
                        <rect x="3" y="9" width="18" height="2" fill="#388e3c"/>
                    </svg>
                </button>
                <input type="text" id="date" readonly>
                <span id="date-display" style="margin-left:0;">No date selected</span>
            </div>
        </div>
        <div id="legend">
            <h3>Legend</h3>
            <table id="legend-table">
                <tr>
                    <td class="emoji">🌱</td>
                    <td><b>Seedling</b><br><span style="font-size:0.95em;">Height &lt; 0.2 m</span></td>
                </tr>
                <tr>
                    <td class="emoji">🌿</td>
                    <td><b>Growing</b><br><span style="font-size:0.95em;">0.2 m ≤ Height &lt; 0.5 m</span></td>
                </tr>
                <tr>
                    <td class="emoji">🌾</td>
                    <td><b>Mature</b><br><span style="font-size:0.95em;">0.5 m ≤ Height &lt; 1.0 m</span></td>
                </tr>
                <tr>
                    <td class="emoji">🪓</td>
                    <td><b>Harvested/Very Tall</b><br><span style="font-size:0.95em;">Height ≥ 1.0 m</span></td>
                </tr>
            </table>
        </div>
    </div>
    <div id="field-visualization"></div>
    <script src="https://unpkg.com/leaflet/dist/leaflet.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/flatpickr"></script>
    <script>
        const locations = {locations_json};
        const municipalities = {municipalities_json};
        const locationColors = {location_colors_json};
        const data = {data_json_str};
        const locationToMunicipality = {{
            "RILLAND": "REIMERSWAAL",
            "EMMELOORD": "NOORDOOSTPOLDER"
        }};

        const map = L.map('map-container').setView([52.5, 5.75], 7.2);
        L.tileLayer('https://{{s}}.tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png', {{
            attribution: '© OpenStreetMap contributors'
        }}).addTo(map);

        fetch('georef-netherlands-gemeente@public.geojson')
            .then(response => response.json())
            .then(geojsonData => {{
                L.geoJSON(geojsonData, {{
                    style: function(feature) {{
                        let munName = feature.properties["gem_name"];
                        if (Array.isArray(munName)) munName = munName[0];
                        if (typeof munName === "string") {{
                            munName = munName.trim().toUpperCase();
                        }} else {{
                            munName = null;
                        }}
                        if (munName && municipalities.map(m => m.trim().toUpperCase()).includes(munName)) {{
                            let loc = Object.keys(locationToMunicipality).find(key => locationToMunicipality[key].trim().toUpperCase() === munName);
                            return {{
                                color: locationColors[loc],
                                weight: 2,
                                fillOpacity: 0.6
                            }};
                        }} else {{
                            return {{
                                color: "#888",
                                weight: 1,
                                fillOpacity: 0.1
                            }};
                        }}
                    }},
                    onEachFeature: function(feature, layer) {{
                        let munName = feature.properties["gem_name"];
                        if (Array.isArray(munName)) munName = munName[0];
                        if (typeof munName === "string") {{
                            munName = munName.trim().toUpperCase();
                        }} else {{
                            munName = null;
                        }}
                        if (munName && municipalities.map(m => m.trim().toUpperCase()).includes(munName)) {{
                            let loc = Object.keys(locationToMunicipality).find(key => locationToMunicipality[key].trim().toUpperCase() === munName);
                            layer.on('click', function() {{
                                showControlsForLocation(loc);
                            }});
                            layer.bindTooltip(munName, {{sticky: true}});
                        }}
                    }}
                }}).addTo(map);
            }});

        let fp = null;
        function showControlsForLocation(location) {{
            document.getElementById('map-container').style.display = 'none';
            document.getElementById('top-bar').style.display = 'flex';
            document.getElementById('back-btn').style.display = 'inline-block';
            document.getElementById('location-title').innerText = location;

            const varietySelect = document.getElementById('variety-select');
            const varieties = [...new Set(data.filter(row => row.LOCATION.toUpperCase() === location).map(row => row.VARIETY))];
            varietySelect.innerHTML = '';
            varieties.forEach(v => {{
                const opt = document.createElement('option');
                opt.value = v;
                opt.text = v;
                varietySelect.appendChild(opt);
            }});

            const dateInput = document.getElementById('date');
            const dateDisplay = document.getElementById('date-display');
            const calendarBtn = document.getElementById('calendar-btn');
            if (fp) fp.destroy();
            const availableDates = [...new Set(data.filter(row => row.LOCATION.toUpperCase() === location && row.VARIETY === varietySelect.value).map(row => row.formatted_date))].sort();
            fp = flatpickr(dateInput, {{
                theme: 'material_green',
                dateFormat: 'Y-m-d',
                minDate: '{min_date}',
                maxDate: '{max_date}',
                enable: availableDates,
                positionElement: calendarBtn,
                onChange: function(selectedDates, dateStr, instance) {{
                    dateDisplay.textContent = dateStr || 'No date selected';
                    updateFieldVisualization(location, varietySelect.value, dateStr);
                }},
            }});
            dateInput.value = '';
            dateDisplay.textContent = 'No date selected';

            varietySelect.onchange = function() {{
                const availableDates = [...new Set(data.filter(row => row.LOCATION.toUpperCase() === location && row.VARIETY === varietySelect.value).map(row => row.formatted_date))].sort();
                fp.set('enable', availableDates);
                dateInput.value = '';
                dateDisplay.textContent = 'No date selected';
                updateFieldVisualization(location, varietySelect.value, '');
            }};
            calendarBtn.onclick = function() {{ fp.open(); }};
            calendarBtn.onkeydown = function(e) {{ if (e.key === 'Enter' || e.key === ' ') fp.open(); }};

            updateFieldVisualization(location, varietySelect.value, '');
        }}

        function getCropImage(height) {{
            if (height === null || isNaN(height)) return '🌱'; // Unknown
            if (height < 0.2) return '🌱'; // Seedling
            if (height < 0.5) return '🌿'; // Growing
            if (height < 1.0) return '🌾'; // Mature
            return '🪓'; // Harvested/very tall
        }}

        function formatNum(val, digits=2) {{
            if (val === null || val === undefined || isNaN(val)) return 'N/A';
            return Number(val).toFixed(digits);
        }}

        function updateFieldVisualization(location, variety, selectedDate) {{
            const filteredData = data.filter(row => {{
                const dateMatch = !selectedDate || row.formatted_date === selectedDate;
                const locationMatch = !location || row.LOCATION.toUpperCase() === location;
                const varietyMatch = !variety || row.VARIETY === variety;
                return dateMatch && locationMatch && varietyMatch;
            }});
            const fieldMap = {{}};
            filteredData.forEach(row => {{
                if (!fieldMap[row.FIELDNUMBER]) {{
                    fieldMap[row.FIELDNUMBER] = row;
                }}
            }});
            const uniqueFields = Object.values(fieldMap).sort((a, b) => (a.FIELDNUMBER || '').localeCompare(b.FIELDNUMBER || ''));
            const container = document.getElementById('field-visualization');
            container.innerHTML = '';
            if (!location || !variety || !selectedDate) {{
                container.innerHTML = '<div>Please select variety, and date.</div>';
                return;
            }}
            if (uniqueFields.length === 0) {{
                container.innerHTML = '<div>No data found for the selected criteria.</div>';
                return;
            }}
            uniqueFields.forEach(row => {{
                const height = parseFloat(row.VegetationHeight_p75);
                const img = getCropImage(height);
                const div = document.createElement('div');
                div.className = 'crop-cell';
                div.innerHTML = `<div class="crop-emoji">${{img}}</div><div class="crop-label">${{row.FIELDNUMBER || ""}}</div>`;
                const tooltip = document.createElement('div');
                tooltip.className = 'tooltip';
                tooltip.innerHTML = `
<b>Field:</b> ${{row.FIELDNUMBER || ''}}<br>
<b>Variety:</b> ${{row.VARIETY || ''}}<br>
<b>Actual Height:</b> <b>${{formatNum(row.VegetationHeight_p75)}}</b> m<br>
<b>Predicted Height:</b> <b>${{formatNum(row.Predicted_Height)}}</b> m<br>
<b>Actual Cover:</b> <b>${{formatNum(row.VegetationCover_perc)}}</b>%<br>
<b>Predicted Cover:</b> <b>${{formatNum(row.Predicted_Cover)}}</b>%<br>
<b>Date:</b> ${{row.formatted_date || ''}}<br>
<b>Location:</b> ${{row.LOCATION || ''}}
`;
                div.appendChild(tooltip);
                container.appendChild(div);
            }});
        }}

        document.getElementById('back-btn').onclick = function() {{
            document.getElementById('map-container').style.display = 'block';
            document.getElementById('top-bar').style.display = 'none';
            document.getElementById('back-btn').style.display = 'none';
            document.getElementById('field-visualization').innerHTML = '';
        }};
    </script>
</body>
</html>
"""

with open('index.html', 'w', encoding='utf-8') as f:
    f.write(html)

import webbrowser
webbrowser.open('index.html')

True